<a href="https://colab.research.google.com/github/dbarrau/TensorFlow_training/blob/main/08_introduction_to_nlp_tensorflow_EXERCISES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Rebuild, compile and train `model_1`, `model_2` and `model_5` using the Keras Sequential API instead of the Functional API.


In [ ]:
!nvidia-smi

Fri Aug 12 16:45:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Download the dataset from Kaggle
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Download and import helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback


# Unzip downloaded data
unzip_data("nlp_getting_started.zip")


--2022-08-12 16:45:14--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-08-12 16:45:15 (136 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]

--2022-08-12 16:45:15--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102

In [ ]:
import pandas as pd
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

## Splitting data into training and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                           train_df_shuffled["target"].to_numpy(),
                                                                           test_size=0.1,
                                                                           random_state=42)

In [ ]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

## Create compile and train model_1 with sequential API

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow as tf

In [ ]:
max_vocab_length = 10000
max_length = 15

text_vectorization = layers.TextVectorization(max_tokens=max_vocab_length,
                             output_mode="int",
                             output_sequence_length=max_length)

text_vectorization.adapt(train_sentences)

In [ ]:
model_1 = models.Sequential([
    layers.Input(shape=(1,), dtype=tf.string),

    text_vectorization,

    layers.Embedding(input_dim=max_vocab_length,
                     output_dim=128,
                     embeddings_initializer="uniform",
                     input_length=max_length,
                     name="embedding_1"),

    layers.GlobalAveragePooling1D(),

    layers.Dense(1, activation="sigmoid")
])

In [ ]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import tensorflow as tf

model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

history_model_1 = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 6s 6ms/step - loss: 0.6110 - accuracy: 0.6916 - val_loss: 0.5378 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4410 - accuracy: 0.8146 - val_loss: 0.4683 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3468 - accuracy: 0.8605 - val_loss: 0.4575 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2839 - accuracy: 0.8897 - val_loss: 0.4691 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2374 - accuracy: 0.9139 - val_loss: 0.4769 - val_accuracy: 0.7822


## Create, compile and train model 2

In [ ]:
model_2 = models.Sequential([
    layers.Input(shape=(1,), dtype=tf.string),
    text_vectorization,

    layers.Embedding(input_dim=max_vocab_length,
                     output_dim=128,
                     embeddings_initializer="uniform",
                     input_length=max_length,
                     name="embedding_1"),
    layers.LSTM(64),
    layers.Dense(1, activation="sigmoid")
])

In [ ]:
model_2.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

history_model_2 = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 7s 10ms/step - loss: 0.5047 - accuracy: 0.7510 - val_loss: 0.4640 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3114 - accuracy: 0.8787 - val_loss: 0.4963 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2185 - accuracy: 0.9199 - val_loss: 0.5849 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1552 - accuracy: 0.9444 - val_loss: 0.6838 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1106 - accuracy: 0.9593 - val_loss: 0.8248 - val_accuracy: 0.7703


## Create compile and train model 5

In [ ]:
model_5 = models.Sequential([
    layers.Input(shape=(1,), dtype=tf.string),
    text_vectorization,
    layers.Embedding(input_dim=max_vocab_length,
                     output_dim=128,
                     embeddings_initializer="uniform",
                     input_length=max_length,
                     name="embedding_1"),
    layers.Conv1D(filters=64,
                  kernel_size=5,
                  strides=1,
                  activation="relu",
                  padding="same"),

    layers.GlobalMaxPool1D(),
    layers.Dense(1, activation="sigmoid")
])

model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_5.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d (Conv1D)             (None, 15, 64)            41024     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,321,089
Trainable params: 1,321,089
No

In [ ]:
history_model_5 = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 9s 6ms/step - loss: 0.5442 - accuracy: 0.7260 - val_loss: 0.4801 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3323 - accuracy: 0.8645 - val_loss: 0.4711 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2007 - accuracy: 0.9270 - val_loss: 0.5600 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1246 - accuracy: 0.9585 - val_loss: 0.6198 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0882 - accuracy: 0.9723 - val_loss: 0.7012 - val_accuracy: 0.7598


# 2. Retrain the baseline model with 10% of the training data. 

How does the model perform compared to the universal sentence encoder model with 10% of the training data?

Get 10% of the data

In [ ]:
train_10_percent_split = int(0.1*len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

# Create tokenization modelling pipeline
model_0 = Pipeline([("tfidf", TfidfVectorizer()),
                     ("clf", MultinomialNB())])

model_0.fit(train_sentences_10_percent,
            train_labels_10_percent)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
model_0.score(val_sentences,val_labels)

0.7020997375328084

# Retrain the baseline model with 10% of the training data. 


In [ ]:
import tensorflow_hub as hub

sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=True,
                                        name="USE")

In [ ]:
model_top = models.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model_top.compile(loss="binary_crossentropy",
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=["accuracy"])

In [ ]:
model_top.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_3 (Dense)             (None, 64)                32832     
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 256,830,721
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history_model_top = model_top.fit(train_sentences,
                                  train_labels,
                                  epochs=5,
                                  validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 25s 86ms/step - loss: 0.4535 - accuracy: 0.7984 - val_loss: 0.4430 - val_accuracy: 0.8058
Epoch 2/5
215/215 [==============================] - 18s 84ms/step - loss: 0.2269 - accuracy: 0.9110 - val_loss: 0.4869 - val_accuracy: 0.8202
Epoch 3/5
215/215 [==============================] - 18s 84ms/step - loss: 0.0739 - accuracy: 0.9768 - val_loss: 0.6067 - val_accuracy: 0.8123
Epoch 4/5
215/215 [==============================] - 18s 84ms/step - loss: 0.0400 - accuracy: 0.9864 - val_loss: 0.6776 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 18s 86ms/step - loss: 0.0269 - accuracy: 0.9904 - val_loss: 0.6418 - val_accuracy: 0.7953


In [ ]:
model_top.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 10ms/step - loss: 0.6418 - accuracy: 0.7953


[0.641756534576416, 0.7952755689620972]

## Retrain the best model you've got so far on the whole training set (no validation split).

Then use this trained model to make predictions on the test dataset and format the predictions into the same format as the sample_submission.csv file from Kaggle (see the Files tab in Colab for what the sample_submission.csv file looks like). Once you've done this, make a submission to the Kaggle competition, how did your model perform?

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):

  """
  Calculates model accuracy, precision, recall and f1 score on a binary classification model.

  Args:

    y_true (array): True labels
    y_pred (array): Predicted labels
  """
  # calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred)

  # Calculate model precision, recal and f1-score using "weighted"
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, 
                                                                               y_pred, 
                                                                               average="weighted")

  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1-score": model_f1}

  return model_results


In [ ]:
model_top_preds = tf.squeeze(tf.round(model_top.predict(val_sentences)))

In [ ]:
model_top_results = calculate_results(y_true=val_labels,
                                      y_pred=model_top_preds)
model_top_results

{'accuracy': 0.7952755905511811,
 'f1-score': 0.7935327770807508,
 'precision': 0.797246356616363,
 'recall': 0.7952755905511811}

# Create fine-tuned USE model

## Set up model checkpoint for fine tuning

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint("fine_tuned_model_checkpoint",
                                   save_weights_only=True,
                                   verbose=0,
                                   monitor="loss",
                                   save_best_only=True)

In [ ]:
model_8 = models.Sequential([
    sentence_encoder_layer,
    layers.Dense(128, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model_8.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_8.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_7 (Dense)             (None, 128)               65664     
                                                                 
 dense_8 (Dense)             (None, 1)                 129       
                                                                 
Total params: 256,863,617
Trainable params: 256,863,617
Non-trainable params: 0
_________________________________________________________________


In [ ]:
starting_epochs = 5

history_model_8 = model_8.fit(train_df[["text"]],
                              train_df[["target"]],
                              epochs=starting_epochs,
                              callbacks=[model_checkpoint])

Epoch 1/5
238/238 [==============================] - 40s 147ms/step - loss: 0.0506 - accuracy: 0.9913
Epoch 2/5
238/238 [==============================] - 34s 142ms/step - loss: 0.0146 - accuracy: 0.9940
Epoch 3/5
238/238 [==============================] - 33s 140ms/step - loss: 0.0110 - accuracy: 0.9949
Epoch 4/5
238/238 [==============================] - 20s 85ms/step - loss: 0.0118 - accuracy: 0.9950
Epoch 5/5
238/238 [==============================] - 34s 143ms/step - loss: 0.0105 - accuracy: 0.9949


In [ ]:
model_8_pred_probs = model_8.predict(test_df.text)

In [ ]:
model_8_preds = tf.squeeze(tf.round(model_8_pred_probs))

In [ ]:
model_8_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 0., 0., 0., 0.], dtype=float32)>

In [ ]:
test_df = pd.read_csv("/content/test.csv")

test_df["target"] = model_8_preds
test_df["target"] = test_df["target"].astype(int)

sample_submission_df = test_df.drop(["text","keyword","location"], axis=1)
sample_submission_df = sample_submission_df.set_index("id")

sample_submission_df.head()


,target
id,
0,1
2,1
3,1
9,1
11,1


In [ ]:
sample_submission_df.to_csv("USE_binary_classification.csv")